# Lab

Using the flights dataframe answer the following questions

1. When do the first and last flights leave each day?

2. When there is a missing value in `dep_time` then this is an indication of canceled flight. Find the number of cancelled flights for each (`carrier`, `origin`) combination.

3. Which carrier has the worst `dep_delay`s? 

4. Which plane (`tailnum`) has the worst on-time record?

5. For each plane, count the number of flights before the first delay of greater than 1 hour.

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/msaricaumbc/DS_data/master/nyc_flights.csv'

flights = pd.read_csv(url)

In [2]:
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


#### Question 1 
When do the first and last flights leave each day?

In [3]:
grouped_flights = flights.groupby("carrier").agg(departue_delay = ('dep_delay','mean'))
grouped_flights

,departue_delay
carrier,
9E,16.725769
AA,8.586016
AS,5.804775
B6,13.022522
DL,9.264505
EV,19.955390
F9,20.215543
FL,18.726075
HA,4.900585


In [4]:
def first_last_flights():
    
    first = flights.dep_time.min()
    last = flights.dep_time.max()
    return first, last
first_last_flights()

(1.0, 2400.0)

In [5]:
flights.iloc[flights["dep_time"].argmax()]

year                              2013
month                               10
day                                 30
dep_time                        2400.0
sched_dep_time                    2359
dep_delay                          1.0
arr_time                         327.0
sched_arr_time                     337
arr_delay                        -10.0
carrier                             B6
flight                             839
tailnum                         N661JB
origin                             JFK
dest                               BQN
air_time                         182.0
distance                          1576
hour                                23
minute                              59
time_hour         2013-10-31T03:00:00Z
Name: 54966, dtype: object

In [6]:
flights.iloc[flights["dep_time"].argmin()]

year                              2013
month                                1
day                                 13
dep_time                           1.0
sched_dep_time                    2249
dep_delay                         72.0
arr_time                         108.0
sched_arr_time                    2357
arr_delay                         71.0
carrier                             B6
flight                              22
tailnum                         N206JB
origin                             JFK
dest                               SYR
air_time                          41.0
distance                           209
hour                                22
minute                              49
time_hour         2013-01-14T03:00:00Z
Name: 10452, dtype: object

#### Question 2
When there is a missing value in dep_time then this is an indication of canceled flight. Find the number of cancelled flights for each (carrier, origin) combination.

In [7]:
cancled_flights = flights[['carrier', 'origin', 'dep_time']]
cancled_flights['dep_time'].isnull().sum()

8255

#### Question 3
Which carrier has the worst dep_delays?

In [8]:
worst_departure_delay = flights[['carrier', 'dep_delay']]
worst_departure_delay

,carrier,dep_delay
0,UA,2.0
1,UA,4.0
2,AA,2.0
3,B6,-1.0
4,DL,-6.0
...,...,...
336771,9E,NaN
336772,9E,NaN
336773,MQ,NaN
336774,MQ,NaN


In [9]:
worst_departure_delay = worst_departure_delay.groupby('carrier').apply(lambda x: x.groupby('carrier').dep_delay.first().sum()).reset_index(name='max_delays')
worst_departure_delay

,carrier,max_delays
0,9E,0.0
1,AA,2.0
2,AS,-1.0
3,B6,-1.0
4,DL,-6.0
5,EV,-3.0
6,F9,-2.0
7,FL,-3.0
8,HA,-3.0
9,MQ,0.0


In [10]:
worst_departure_delay.loc[worst_departure_delay['max_delays'].idxmax()]

carrier         OO
max_delays    67.0
Name: 10, dtype: object

#### Question 4
Which plane (tailnum) has the worst on-time record?

In [11]:
worst_ontime_record = flights[['tailnum', 'dep_delay', 'arr_delay']]

worst_ontime_record = worst_ontime_record.groupby('tailnum').apply(lambda x: x.groupby('arr_delay').dep_delay.first().sum()).reset_index(name='max_delays')

In [12]:
worst_ontime_record

,tailnum,max_delays
0,D942DN,126.0
1,N0EGMQ,3130.0
2,N10156,2200.0
3,N102UW,413.0
4,N103US,-85.0
...,...,...
4038,N997DL,497.0
4039,N998AT,789.0
4040,N998DL,1739.0
4041,N999DN,1172.0


In [13]:
worst_ontime_record.loc[worst_ontime_record['max_delays'].idxmax()]

tailnum       N15980
max_delays    6555.0
Name: 241, dtype: object

#### Question 5
For each plane, count the number of flights before the first delay of greater than 1 hour.

In [14]:
Count_per_flights = flights[['tailnum', 'dep_delay', 'arr_delay']]
Count_per_flights

,tailnum,dep_delay,arr_delay
0,N14228,2.0,11.0
1,N24211,4.0,20.0
2,N619AA,2.0,33.0
3,N804JB,-1.0,-18.0
4,N668DN,-6.0,-25.0
...,...,...,...
336771,NaN,NaN,NaN
336772,NaN,NaN,NaN
336773,N535MQ,NaN,NaN
336774,N511MQ,NaN,NaN


In [15]:
Count_per_flights

,tailnum,dep_delay,arr_delay
0,N14228,2.0,11.0
1,N24211,4.0,20.0
2,N619AA,2.0,33.0
3,N804JB,-1.0,-18.0
4,N668DN,-6.0,-25.0
...,...,...,...
336771,NaN,NaN,NaN
336772,NaN,NaN,NaN
336773,N535MQ,NaN,NaN
336774,N511MQ,NaN,NaN


In [16]:
delay_less_than_one_for_all_flights_count = flights[(flights.dep_delay <= 1.0) & (flights.arr_delay <= 1.0)]["tailnum"].value_counts()

In [17]:
delay_less_than_one_for_all_flights_count

N725MQ    325
N722MQ    302
N723MQ    279
N711MQ    269
N713MQ    255
         ... 
N831AW      1
N829MH      1
N7BGAA      1
N828MH      1
N841MH      1
Name: tailnum, Length: 3883, dtype: int64